foo.037	Maize, Rice, Soybeans, and Wheat Yield Trends

https://s3-us-west-2.amazonaws.com/earthstat/earthstat/wp-content/uploads/filebase/yieldtrends/yt_geotiff.zip

File type:geotiff

In [21]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3

In [22]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/foo_037_crops_Yield_Trends/"

s3_file1 = "foo_037_crops_Yield_Trends_maize_categorical.tif"
s3_file2 = "foo_037_crops_Yield_Trends_rice_categorical.tif"
s3_file3 = "foo_037_crops_Yield_Trends_soybean_categorical.tif"
s3_file4 = "foo_037_crops_Yield_Trends_wheat_categorical.tif"
s3_file5 = "foo_037_crops_Yield_Trends_maize_percent.tif"
s3_file6 = "foo_037_crops_Yield_Trends_rice_percent.tif"
s3_file7 = "foo_037_crops_Yield_Trends_soybean_percent.tif"
s3_file8 = "foo_037_crops_Yield_Trends_wheat_percent.tif"

s3_key_orig1 = s3_folder + s3_file1
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + s3_file2
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

s3_key_orig3 = s3_folder + s3_file3
s3_key_edit3 = s3_key_orig3[0:-4] + "_edit.tif"

s3_key_orig4 = s3_folder + s3_file4
s3_key_edit4 = s3_key_orig4[0:-4] + "_edit.tif"

s3_key_orig5 = s3_folder + s3_file5
s3_key_edit5 = s3_key_orig5[0:-4] + "_edit.tif"

s3_key_orig6= s3_folder + s3_file6
s3_key_edit6 = s3_key_orig6[0:-4] + "_edit.tif"

s3_key_orig7 = s3_folder + s3_file7
s3_key_edit7 = s3_key_orig7[0:-4] + "_edit.tif"

s3_key_orig8 = s3_folder + s3_file8
s3_key_edit8 = s3_key_orig8[0:-4] + "_edit.tif"

band_ids_category = ["maize","rice","soybean", "wheat"]
band_ids_percent = ["maize","rice","soybean", "wheat"]
merge_name_1 = "foo_037_crops_Yield_Trends_categorical.tif"
merge_name_2 = "foo_037_crops_Yield_Trends_percent.tif"
s3_key_merge_1 = s3_folder + merge_name_1
s3_key_merge_2 = s3_folder + merge_name_2

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [23]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/foo.037/yt_geotiff/"

file_name1 = "categorical_maize.tif"
file_name2 = "categorical_rice.tif"
file_name3 = "categorical_soybean.tif"
file_name4 = "categorical_wheat.tif"
file_name5 = "percentage_maize.tif"
file_name6 = "percentage_rice.tif"
file_name7 = "percentage_soy.tif"
file_name8 = "percentage_wheat.tif"

local_orig1 = local_folder + file_name1
local_orig2 = local_folder + file_name2
local_orig3 = local_folder + file_name3
local_orig4 = local_folder + file_name4
local_orig5 = local_folder + file_name5
local_orig6 = local_folder + file_name6
local_orig7 = local_folder + file_name7
local_orig8 = local_folder + file_name8

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "edit.tif" 
local_edit3 = local_orig3[:-orig_extension_length] + "edit.tif" 
local_edit4 = local_orig4[:-orig_extension_length] + "edit.tif" 
local_edit5 = local_orig5[:-orig_extension_length] + "edit.tif" 
local_edit6 = local_orig6[:-orig_extension_length] + "edit.tif" 
local_edit7 = local_orig7[:-orig_extension_length] + "edit.tif" 
local_edit8 = local_orig8[:-orig_extension_length] + "edit.tif" 



merge_files_1 = [local_edit1, local_edit2, local_edit3, local_edit4]
merge_files_2 = [local_edit5, local_edit6, local_edit7, local_edit8]
tmp_merge_1 = local_folder + merge_name_1
tmp_merge_2 = local_folder + merge_name_2

Use rasterio to reproject and compress

In [24]:
files = [local_orig1, local_orig2, local_orig3, local_orig4, local_orig5, local_orig6, local_orig7, local_orig8]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.0833333333333286, 0.0, -180.0,
       0.0, -0.0833333333333286, 89.99999999998977), 'blockxsize': 432, 'blockysize': 224, 'tiled': True, 'compress': 'packbits', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.0833333333333286, 0.0, -180.0,
       0.0, -0.0833333333333286, 89.99999999998977), 'blockxsize': 432, 'blockysize': 224, 'tiled': True, 'compress': 'packbits', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.0833333333333286, 0.0, -180.0,
       0.0, -0.0833333333333286, 89.99999999998977), 'blockxsize': 432, 'blockysize': 224, 'tiled': True, 'compress': 'packbits', 'inte

In [25]:
# Note - this is the core of Vizz's netcdf2tif function
def convert_asc_to_tif(orig_name, edit_name):

    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example

        data = src.read()[0]

        rows = data.shape[0]
        columns = data.shape[1]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = None
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }

        with rio.open(edit_name, "w", **profile) as dst:
            dst.write(data.astype(profile["dtype"]), 1)

In [26]:
convert_asc_to_tif(local_orig1, local_edit1)
convert_asc_to_tif(local_orig2, local_edit2)
convert_asc_to_tif(local_orig3, local_edit3)
convert_asc_to_tif(local_orig4, local_edit4)
convert_asc_to_tif(local_orig5, local_edit5)
convert_asc_to_tif(local_orig6, local_edit6)
convert_asc_to_tif(local_orig7, local_edit7)
convert_asc_to_tif(local_orig8, local_edit8)

2160
4320
2160
4320
2160
4320
2160
4320
2160
4320
2160
4320
2160
4320
2160
4320


In [27]:
with rio.open(merge_files_1[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files_1)
)
    
with rio.open(tmp_merge_1, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files_1):
        print(idx)
        with rio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)
                

    

0
1
2
3


In [30]:
with rio.open(merge_files_2[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files_2)
)
    
with rio.open(tmp_merge_2, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files_2):
        print(idx)
        with rio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)
                

0
1
2
3


Upload orig and edit files to s3

In [28]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))
s3_upload.upload_file(local_orig4, s3_bucket, s3_key_orig4,
                         Callback=ProgressPercentage(local_orig4))
s3_upload.upload_file(local_orig5, s3_bucket, s3_key_orig5,
                         Callback=ProgressPercentage(local_orig5))
s3_upload.upload_file(local_orig6, s3_bucket, s3_key_orig6,
                         Callback=ProgressPercentage(local_orig6))
s3_upload.upload_file(local_orig7, s3_bucket, s3_key_orig7,
                         Callback=ProgressPercentage(local_orig7))
s3_upload.upload_file(local_orig8, s3_bucket, s3_key_orig8,
                         Callback=ProgressPercentage(local_orig8))

# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit2, s3_bucket, s3_key_edit2,
                         Callback=ProgressPercentage(local_edit2))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))
s3_upload.upload_file(local_edit4, s3_bucket, s3_key_edit4,
                         Callback=ProgressPercentage(local_edit4))
s3_upload.upload_file(local_edit5, s3_bucket, s3_key_edit5,
                         Callback=ProgressPercentage(local_edit5))
s3_upload.upload_file(local_edit6, s3_bucket, s3_key_edit6,
                         Callback=ProgressPercentage(local_edit6))
s3_upload.upload_file(local_edit7, s3_bucket, s3_key_edit7,
                         Callback=ProgressPercentage(local_edit7))
s3_upload.upload_file(local_edit8, s3_bucket, s3_key_edit8,
                         Callback=ProgressPercentage(local_edit8))

# Merge Files

s3_upload.upload_file(tmp_merge_1, s3_bucket, s3_key_merge_1,
           Callback=ProgressPercentage(tmp_merge_1))
s3_upload.upload_file(tmp_merge_2, s3_bucket, s3_key_merge_2,
           Callback=ProgressPercentage(tmp_merge_2))


/Users/Max81007/Desktop/Python/Resource_Watch/Raster/foo.037/yt_geotiff/foo_037_crops_Yield_Trends_percent.tif  11841616 / 11841616.0  (100.00%)%)

In [31]:
os.environ["Zs3_key_1"] = "s3://wri-public-data/" + s3_key_merge_1 
os.environ["Zs3_key_inspect_1"] = "wri-public-data/" + s3_key_merge_1  

os.environ["Zs3_key_2"] = "s3://wri-public-data/" + s3_key_merge_2 
os.environ["Zs3_key_inspect_2"] = "wri-public-data/" + s3_key_merge_2  

In [32]:
os.environ["Zgs_key_1"] = "gs://resource-watch-public/" + s3_key_merge_1
os.environ["Zgs_key_2"] = "gs://resource-watch-public/" + s3_key_merge_2

In [33]:
!echo %Zs3_key_inspect_1%
!aws s3 ls %Zs3_key_inspect_1%

!echo %Zs3_key_inspect_2%
!aws s3 ls %Zs3_key_inspect_2%

wri-public-data/resourcewatch/raster/foo_037_crops_Yield_Trends/foo_037_crops_Yield_Trends_categorical.tif
2017-09-28 14:27:45    9282505 foo_037_crops_Yield_Trends_categorical.tif
wri-public-data/resourcewatch/raster/foo_037_crops_Yield_Trends/foo_037_crops_Yield_Trends_percent.tif
2017-09-28 14:27:46   11841616 foo_037_crops_Yield_Trends_percent.tif


In [34]:
!gsutil cp %Zs3_key_1% %Zgs_key_1%
!gsutil cp %Zs3_key_2% %Zgs_key_2%

Non-MD5 etag ("2e7eaf019f8da9126269a0f6ad8cab6f-2") present for key <Key: wri-public-data,resourcewatch/raster/foo_037_crops_Yield_Trends/foo_037_crops_Yield_Trends_categorical.tif>, data integrity checks are not possible.
Copying s3://wri-public-data/resourcewatch/raster/foo_037_crops_Yield_Trends/foo_037_crops_Yield_Trends_categorical.tif [Content-Type=binary/octet-stream]...
/ [0 files][    0.0 B/  8.9 MiB]                                                
-
- [0 files][  5.7 MiB/  8.9 MiB]                                                
\
\ [1 files][  8.9 MiB/  8.9 MiB]                                                

Operation completed over 1 objects/8.9 MiB.                                      
Non-MD5 etag ("a747243df8768ca7d2f21e9bfa82b85f-2") present for key <Key: wri-public-data,resourcewatch/raster/foo_037_crops_Yield_Trends/foo_037_crops_Yield_Trends_percent.tif>, data integrity checks are not possible.
Copying s3://wri-public-data/resourcewatch/raster/foo_037_crops_Yield_

In [35]:
os.environ["asset_id_1"] = "users/resourcewatch/foo_037_crops_yield_trends_categorical"
!earthengine upload image --asset_id=%asset_id_1% %Zgs_key_1%

os.environ["asset_id_2"] = "users/resourcewatch/foo_037_crops_yield_trends_percent"
!earthengine upload image --asset_id=%asset_id_2% %Zgs_key_2%

Started upload task with ID: IQ77CCV6GJZ3GQAEFPQIDEUF
Started upload task with ID: RZTZEWHC262U545ZKTF2OSN7


In [44]:
os.environ["band_names"] = str(band_ids_category)
!earthengine asset set -p band_names="%band_names%" %asset_id_1%



In [45]:
os.environ["band_names"] = str(band_ids_percent)
!earthengine asset set -p band_names="%band_names%" %asset_id_2%